In [41]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd

- Pclass Passenger Class(1 = 1st; 2 = 2nd, 3 = 3rd
- survival Survival (0 = No; 1 = Yes)
- name Name
- sex Sex
- age Age
- sibsp Number of Siblings/Spouse Aboard
- parch Number of Parents/Children Aboard
- ticket Ticket Number
- fare Passenger Fare (British pound)
- cabin Cabin
- embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
- boat Lifeboat
- body Body Identification Number
- home.dest Home/Destination

In [42]:
df = pd.read_excel(r'D:\Ugo\Practice ML\ML Datasets\Titanic Dataset\titanic3.xls')
original_df = pd.DataFrame.copy(df)
 
df.drop(['body', 'name'], 1, inplace=True)
df.convert_objects(convert_numeric=True)
df.head()
df.fillna(0, inplace=True)

def handle_non_numerical_data(df):
    
    # handling non_numerical data: must convert.
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        # print(column, df[column].dtype)
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            
            column_contents = df[column].values.tolist()
            # finding just the uniques
            unique_elements = set(column_contents)
            # great, found them.
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    # creating dict that contains new
                    # id per unique string
                    text_digit_vals[unique] = x
                    x += 1
            
            # now we map the new "id" value
            # to replace the string.
            df[column] = list(map(convert_to_int, df[column]))
            
    return df

df = handle_non_numerical_data(df)
df.head()

# add/remove features just to see what impact they have
df.drop(['ticket', 'home.dest'], 1, inplace=True)

X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

clf = MeanShift()
clf.fit(X)


C:\Users\DAMILOLA\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """


MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [43]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]
n_clusters_ = len(np.unique(labels))
    
    
survival_rates = {}
for i in range(n_clusters_):
    temp_df = original_df[ (original_df['cluster_group'] == float(i)) ]
    survival_cluster = temp_df[ (temp_df['survived'] == 1) ]
    survival_rate = len(survival_cluster)/ len(temp_df)
    survival_rates[i] = survival_rate
    
print(survival_rates)

C:\Users\DAMILOLA\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


{0: 0.37209302325581395, 1: 1.0, 2: 0.6458333333333334, 3: 0.1}


In [44]:
print(original_df[ (original_df['cluster_group']==0) ])

      pclass  survived                                              name  \
0          1         1                     Allen, Miss. Elisabeth Walton   
5          1         1                               Anderson, Mr. Harry   
6          1         1                 Andrews, Miss. Kornelia Theodosia   
7          1         0                            Andrews, Mr. Thomas Jr   
8          1         1     Appleton, Mrs. Edward Dale (Charlotte Lamson)   
9          1         0                           Artagaveytia, Mr. Ramon   
12         1         1                     Aubart, Mme. Leontine Pauline   
13         1         1                      Barber, Miss. Ellen "Nellie"   
14         1         1              Barkworth, Mr. Algernon Henry Wilson   
15         1         0                               Baumann, Mr. John D   
18         1         1                             Bazzani, Miss. Albina   
19         1         0                              Beattie, Mr. Thomson   
20         1

In [45]:
print(original_df[ (original_df['cluster_group']==0) ].describe())

            pclass     survived         age       sibsp        parch  \
count  1247.000000  1247.000000  987.000000  1247.00000  1247.000000   
mean      2.343224     0.372093   29.456180     0.48757     0.304731   
std       0.812769     0.483557   14.234827     1.04994     0.657224   
min       1.000000     0.000000    0.166700     0.00000     0.000000   
25%       2.000000     0.000000   21.000000     0.00000     0.000000   
50%       3.000000     0.000000   28.000000     0.00000     0.000000   
75%       3.000000     1.000000   38.000000     1.00000     0.000000   
max       3.000000     1.000000   80.000000     8.00000     4.000000   

              fare        body  cluster_group  
count  1246.000000  113.000000         1247.0  
mean     25.238028  162.123894            0.0  
std      28.152057   98.916644            0.0  
min       0.000000    1.000000            0.0  
25%       7.895800   70.000000            0.0  
50%      13.458350  166.000000            0.0  
75%      27.862

In [46]:
print(original_df[ (original_df['cluster_group']==1) ])

     pclass  survived                                               name  \
49        1         1                 Cardeza, Mr. Thomas Drake Martinez   
50        1         1  Cardeza, Mrs. James Warburton Martinez (Charlo...   
183       1         1                             Lesurer, Mr. Gustave J   
302       1         1                                   Ward, Miss. Anna   

        sex   age  sibsp  parch    ticket      fare        cabin embarked  \
49     male  36.0      0      1  PC 17755  512.3292  B51 B53 B55        C   
50   female  58.0      0      1  PC 17755  512.3292  B51 B53 B55        C   
183    male  35.0      0      0  PC 17755  512.3292         B101        C   
302  female  35.0      0      0  PC 17755  512.3292          NaN        C   

    boat  body                                       home.dest  cluster_group  
49     3   NaN  Austria-Hungary / Germantown, Philadelphia, PA            1.0  
50     3   NaN                    Germantown, Philadelphia, PA           

In [47]:
print(original_df[ (original_df['cluster_group']==1) ].describe())

       pclass  survived        age  sibsp    parch      fare  body  \
count     4.0       4.0   4.000000    4.0  4.00000    4.0000   0.0   
mean      1.0       1.0  41.000000    0.0  0.50000  512.3292   NaN   
std       0.0       0.0  11.343133    0.0  0.57735    0.0000   NaN   
min       1.0       1.0  35.000000    0.0  0.00000  512.3292   NaN   
25%       1.0       1.0  35.000000    0.0  0.00000  512.3292   NaN   
50%       1.0       1.0  35.500000    0.0  0.50000  512.3292   NaN   
75%       1.0       1.0  41.500000    0.0  1.00000  512.3292   NaN   
max       1.0       1.0  58.000000    0.0  1.00000  512.3292   NaN   

       cluster_group  
count            4.0  
mean             1.0  
std              0.0  
min              1.0  
25%              1.0  
50%              1.0  
75%              1.0  
max              1.0  


In [48]:
print(original_df[ (original_df['cluster_group']==2) ])

     pclass  survived                                               name  \
1         1         1                     Allison, Master. Hudson Trevor   
2         1         0                       Allison, Miss. Helen Loraine   
3         1         0               Allison, Mr. Hudson Joshua Creighton   
4         1         0    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
10        1         0                             Astor, Col. John Jacob   
11        1         1  Astor, Mrs. John Jacob (Madeleine Talmadge Force)   
16        1         0                           Baxter, Mr. Quigg Edmond   
17        1         1    Baxter, Mrs. James (Helene DeLaudeniere Chaput)   
23        1         1                              Bidois, Miss. Rosalie   
24        1         1                                  Bird, Miss. Ellen   
32        1         1                            Bonnell, Miss. Caroline   
35        1         1                           Bowen, Miss. Grace Scott   
66        1 

In [49]:
print(original_df[ (original_df['cluster_group']==2) ].describe())

       pclass   survived        age      sibsp      parch        fare  \
count    48.0  48.000000  47.000000  48.000000  48.000000   48.000000   
mean      1.0   0.645833  36.157802   0.770833   1.291667  200.573958   
std       0.0   0.483321  17.180574   0.904824   1.071057   60.086908   
min       1.0   0.000000   0.916700   0.000000   0.000000   49.500000   
25%       1.0   0.000000  24.000000   0.000000   0.000000  151.550000   
50%       1.0   1.000000  36.000000   1.000000   1.000000  216.639600   
75%       1.0   1.000000  49.000000   1.000000   2.000000  262.375000   
max       1.0   1.000000  67.000000   3.000000   4.000000  263.000000   

             body  cluster_group  
count    6.000000           48.0  
mean   111.500000            2.0  
std     36.719205            0.0  
min     45.000000            2.0  
25%    102.500000            2.0  
50%    123.000000            2.0  
75%    132.250000            2.0  
max    147.000000            2.0  


In [50]:
print(original_df[ (original_df['cluster_group']==3) ])

      pclass  survived                                               name  \
629        3         0                        Andersson, Mr. Anders Johan   
632        3         0  Andersson, Mrs. Anders Johan (Alfrida Konstant...   
644        3         0         Asplund, Mr. Carl Oscar Vilhelm Gustafsson   
646        3         1  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
831        3         0                     Goodwin, Mr. Charles Frederick   
832        3         0            Goodwin, Mrs. Frederick (Augusta Tyler)   
1106       3         0             Panula, Mrs. Juha (Maria Emilia Ojala)   
1146       3         0               Rice, Mrs. William (Margaret Norton)   
1179       3         0                              Sage, Mr. John George   
1180       3         0                     Sage, Mrs. John (Annie Bullen)   

         sex   age  sibsp  parch    ticket     fare cabin embarked boat  \
629     male  39.0      1      5    347082  31.2750   NaN        S  NaN   
63

In [51]:
print(original_df[ (original_df['cluster_group']==3) ].describe())

       pclass   survived        age      sibsp      parch       fare  \
count    10.0  10.000000   8.000000  10.000000  10.000000  10.000000   
mean      3.0   0.100000  39.875000   0.800000   6.000000  42.703750   
std       0.0   0.316228   1.552648   0.421637   1.632993  15.590194   
min       3.0   0.000000  38.000000   0.000000   5.000000  29.125000   
25%       3.0   0.000000  39.000000   1.000000   5.000000  31.303125   
50%       3.0   0.000000  39.500000   1.000000   5.000000  35.537500   
75%       3.0   0.000000  40.250000   1.000000   6.000000  46.900000   
max       3.0   1.000000  43.000000   1.000000   9.000000  69.550000   

             body  cluster_group  
count    2.000000           10.0  
mean   234.500000            3.0  
std    130.814755            0.0  
min    142.000000            3.0  
25%    188.250000            3.0  
50%    234.500000            3.0  
75%    280.750000            3.0  
max    327.000000            3.0  


In [53]:
cluster_0 = original_df[ (original_df['cluster_group'] == 0) ]
cluster_0_fe = cluster_0[ (cluster_0['pclass'] == 1)]
cluster_0_fe.describe()

,pclass,survived,age,sibsp,parch,fare,body,cluster_group
count,271.0,271.000000,233.000000,271.000000,271.000000,271.000000,29.000000,271.0
mean,1.0,0.608856,39.733906,0.383764,0.199262,61.212315,173.448276,0.0
std,0.0,0.488909,13.991685,0.523834,0.476250,38.069434,85.866835,0.0
min,1.0,0.000000,4.000000,0.000000,0.000000,0.000000,16.000000,0.0
25%,1.0,0.000000,29.000000,0.000000,0.000000,29.700000,110.000000,0.0
50%,1.0,1.000000,39.000000,0.000000,0.000000,53.100000,175.000000,0.0
75%,1.0,1.000000,50.000000,1.000000,0.000000,79.650000,245.000000,0.0
max,1.0,1.000000,80.000000,2.000000,2.000000,227.525000,307.000000,0.0


This means that 60 percent of the passengers survived